In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import gc
import os
import time
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# --------------------
# Reproducibility Settings
# --------------------
def set_seed(seed=42):
    """GPU使用時でも再現性を保証する設定"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # multi-GPU対応
    
    # CuDNNの決定論的動作を有効化
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Python hash seedも固定（環境変数として設定推奨）
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# --------------------
# Paths
# --------------------
SUBMISSION_DIR = "/kaggle/working"
os.makedirs(SUBMISSION_DIR, exist_ok=True)
BEST_MODEL_PATH = os.path.join(SUBMISSION_DIR, "best_bilstm_stream.pth")

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# --------------------
# Hyperparameters
# --------------------
patch_size = 150_000
N_sub_patches = 150

# ★ メモリ対策: データを小分けにするサイズ (500万行ずつ処理)
CHUNK_SIZE = 50_000_000 
overlap_rate = 0.2

# --------------------
# Helper Function: Feature Extraction
# --------------------
def extract_features(x, N_sub):
    # x: (patch_size,) -> float32
    L_sub = len(x) // N_sub
    x = x[:N_sub * L_sub].reshape(N_sub, L_sub)
    
    # 高速化のため float32 で計算
    mean = x.mean(axis=1)
    std = x.std(axis=1)
    min_ = x.min(axis=1)
    max_ = x.max(axis=1)
    q25 = np.quantile(x, 0.25, axis=1)
    q50 = np.quantile(x, 0.50, axis=1)
    q75 = np.quantile(x, 0.75, axis=1)
    iqr = q75 - q25
    
    return np.stack([mean, std, min_, max_, q25, q50, q75, iqr], axis=1)

# --------------------
# Main: Streaming Processing (ここが心臓部)
# --------------------
rootpath = "../input/LANL-Earthquake-Prediction/"
csv_path = rootpath + "train.csv"

# 結果を格納するリスト
all_X = []
all_Y = []

# 正規化用のパラメータ（初期値）
train_mean = 0
train_std = 1
target_min = 0
target_max = 1

# CSVを「イテレータ」として開く（まだデータは読み込まれない）
# dtypeを指定してメモリ節約
reader = pd.read_csv(
    csv_path, 
    chunksize=CHUNK_SIZE,
    dtype={"acoustic_data": np.int16, "time_to_failure": np.float32}
)

print("Starting Chunk Processing...")

for chunk_idx, df_chunk in enumerate(tqdm(reader)):
    
    # 波形とターゲットを取得
    x_chunk = df_chunk["acoustic_data"].values.astype(np.float32)
    y_chunk = df_chunk["time_to_failure"].values.astype(np.float32)
    
    # -------------------------------------------------------
    # 最初のチャンクを使って、正規化パラメータを決める（近似）
    # 全データをなめると時間がかかるため、最初の5000万行で推定する
    # -------------------------------------------------------
    if chunk_idx == 0:
        train_mean = x_chunk.mean()
        train_std = x_chunk.std()
        target_min = y_chunk.min()
        target_max = y_chunk.max() # 注: 全体のmaxではないが、実用上大きな問題ではない
        print(f"Stats estimated: Mean={train_mean:.4f}, Std={train_std:.4f}")

    # 正規化 (In-place)
    x_chunk -= train_mean
    x_chunk /= train_std
    
    # パッチ生成ループ
    # チャンクごとに処理するため、チャンクの継ぎ目のデータは捨てることになるが
    # データ量が膨大なので学習への影響は軽微として無視する
    
    step = int(patch_size * (1 - overlap_rate))
    
    chunk_feats = []
    chunk_targets = []
    
    for i in range(0, len(x_chunk) - patch_size, step):
        # ターゲット（パッチの最後）
        y_val = y_chunk[i + patch_size]
        
        # 地震発生直後の不連続点を含むパッチを除外
        # (パッチの最後の時間が、開始時間より増えていたらリセットが起きている)
        if y_val > y_chunk[i]:
            continue
            
        # 特徴量抽出
        x_raw = x_chunk[i : i + patch_size]
        feats = extract_features(x_raw, N_sub_patches)
        
        chunk_feats.append(feats)
        chunk_targets.append(y_val)
    
    # 結果をリストに追加
    if len(chunk_feats) > 0:
        all_X.append(np.array(chunk_feats))
        all_Y.append(np.array(chunk_targets))
    
    # メモリ解放！
    del df_chunk, x_chunk, y_chunk, chunk_feats, chunk_targets
    gc.collect()

print("Processing complete. Concatenating...")

# リストを結合して1つのnumpy配列にする
# ここでメモリを使うが、特徴量化されているためサイズは元の1/1000以下
X_final = np.concatenate(all_X, axis=0).astype(np.float32)
Y_final = np.concatenate(all_Y, axis=0).astype(np.float32)

# Yの正規化 (MinMax)
Y_final = (Y_final - target_min) / (target_max - target_min)

print(f"Final Data Shape: X={X_final.shape}, Y={Y_final.shape}")
print(f"Memory Usage of X: {X_final.nbytes / 1024**2:.2f} MB")

In [ ]:
batch_size = 32
num_epochs = 1000 # 特徴量が軽いのでエポック回せます
learning_rate = 1e-3
hidden_size = 128
num_layers = 2

# --------------------
# Dataset & Model (軽量版)
# --------------------
class SimpleDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

# DataLoader用のworker初期化関数（再現性のため）
def worker_init_fn(worker_id):
    """各DataLoaderワーカーでシードを固定"""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Split
indices = np.arange(len(X_final))
np.random.seed(42)
np.random.shuffle(indices)
split = int(0.9 * len(indices))

train_ds = SimpleDataset(X_final[indices[:split]], Y_final[indices[:split]])
valid_ds = SimpleDataset(X_final[indices[split:]], Y_final[indices[split:]])

train_loader = DataLoader(
    train_ds, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=2,
    worker_init_fn=worker_init_fn,
    generator=torch.Generator().manual_seed(42)  # シャッフル用のジェネレータ
)
valid_loader = DataLoader(
    valid_ds, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=2,
    worker_init_fn=worker_init_fn
)

# モデル定義
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :]).squeeze(1)
        return out

model = BiLSTM(8, hidden_size, num_layers).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
criterion = nn.L1Loss()

# Training Loop
best_val = float("inf")

print("Start Training...")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_ds)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            val_loss += criterion(pred, y).item() * x.size(0)
    val_loss /= len(valid_ds)
    
    print(f"Epoch {epoch+1}/{num_epochs} | Train: {train_loss:.4f} | Valid: {val_loss:.4f}")
    
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), BEST_MODEL_PATH)

print("Done.")

In [ ]:
# --------------------
# Inference
# --------------------
import os
import pandas as pd
import numpy as np
import torch

# モデルの準備
model = BiLSTM(8, hidden_size, num_layers).to(device)
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

# 提出用ファイルの読み込み
sample_submission = pd.read_csv(rootpath + "sample_submission.csv")
SUBMISSION_PATH = "/kaggle/working/submission.csv"

# テストデータの読み込みと予測
test_preds = []

print("Starting Inference...")

# ★重要: sample_submissionの順序通りにファイルを読み込む
for seg_id in tqdm(sample_submission["seg_id"]):
    # ファイルパスを作成
    file_path = os.path.join(rootpath, "test", seg_id + ".csv")
    
    # 読み込み
    df = pd.read_csv(file_path, dtype={"acoustic_data": np.int16})
    x = df["acoustic_data"].values.astype(np.float32)
    
    # 1. 正規化 (学習時と同じパラメータを使用！)
    # 先ほどのコードで計算した train_mean, train_std を使う
    # 変数が残っていない場合は、学習ログを見て手動で代入してください (例: train_mean = 4.51...)
    x -= train_mean 
    x /= train_std
    
    # 2. 特徴量抽出 (学習時と同じ関数を使用)
    # extract_features関数は学習コード内で定義したものを使います
    feats = extract_features(x, N_sub_patches) # shape: (25, 8)
    
    # 3. テンソル化 (バッチ次元を追加: [1, 25, 8])
    x_tensor = torch.tensor(feats).unsqueeze(0).to(device)
    
    # 4. 予測
    with torch.no_grad():
        pred = model(x_tensor).item()
    
    test_preds.append(pred)

# 結果を格納
# Yの逆正規化 (学習時と同じ min/max を使用)
test_preds = np.array(test_preds)
test_preds = test_preds * (target_max - target_min) + target_min

sample_submission["time_to_failure"] = test_preds
sample_submission.to_csv(SUBMISSION_PATH, index=False)

print(f"Submission saved to: {SUBMISSION_PATH}")
print(sample_submission.head())